In [ ]:
! pip install langchain_community langchain-huggingface langchain -q

In [ ]:
! pip install faiss-cpu -q

In [ ]:
import faiss
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_huggingface import HuggingFaceEndpoint
from langchain.text_splitter import CharacterTextSplitter
import numpy as np

In [ ]:
with open('the-4-vedas.txt','r',encoding='utf-8') as reader:
  text = reader.read()
  reader.close()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_text(text)

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


<ipython-input-5-49a97340da10>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
embeddings = hf_embeddings.embed_documents(chunks)
dimension = len(embeddings[0])

In [ ]:
quantizer = faiss.IndexFlatL2(dimension)
index = faiss.IndexIVFFlat(quantizer, dimension, 100)
index.train(np.array(embeddings).astype('float32'))
index.add(np.array(embeddings).astype('float32'))

In [ ]:
# vector_store = FAISS.from_texts(
#     chunks,
#     embedding=hf_embeddings,
# )
from langchain.schema import Document

documents = [Document(page_content=chunk) for chunk in chunks]
docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})
index_to_docstore_id = {i: str(i) for i in range(len(documents))}

vector_store = FAISS(embedding_function=hf_embeddings.embed_query,index=index, docstore=docstore, index_to_docstore_id=index_to_docstore_id)

# Step 4: Save index
vector_store.save_local('vector_search')

In [ ]:
from langchain.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id='mistralai/Mistral-7B-Instruct-v0.2',
    task='text-generation',
    temperature=0.6
)


<ipython-input-6-357ee372a590>:3: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(


In [ ]:
query = "what is Yantu Vishwataha"
vector_ser = FAISS.load_local("/content/vector_search",embeddings=hf_embeddings,allow_dangerous_deserialization=True)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vector_ser.as_retriever())

answer = qa({"query": query})
print(answer)

<ipython-input-8-0482b08b8b34>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa({"query": query})
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


{'query': 'what is Yantu Vishwataha', 'result': " Yantu Vishwataha is a phrase used in the Rigveda mantras, which can be translated to mean 'Let noble thoughts come from every side'. It is a common invocation used in the mantras to invoke blessings and positive energies."}
